In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import pymongo
import json
import sys
from pymongo import MongoClient as Connection

In [2]:
connection = Connection('localhost',27017)
db = connection.ecobici #Crea la base Ecobicis

In [3]:
estaciones = pd.read_csv('estaciones.csv')
data = estaciones

In [7]:
data_json = json.loads(data.to_json(orient='records')) #Convierte de csv a json
db.bici.insert_many(data_json) #Crea colleccion bicis e inserta datos json a mongo

In [8]:
#Encontrar el rango de coordenadas para hacer un cluster
min_lat = db.bici.find_one(sort=[("lat", 1)])["lat"]
max_lat =  db.bici.find_one(sort=[("lat", -1)])["lat"]
min_lon = db.bici.find_one(sort=[("lon", 1)])["lon"]
max_lon =  db.bici.find_one(sort=[("lon", -1)])["lon"]

In [9]:
print(min_lat, max_lat, min_lon, max_lon)

19.35827 19.444031 -99.207808 -99.130918


In [10]:
cen_lat = (min_lat + max_lat) /2
cen_lon = (min_lon + max_lon) /2
print(cen_lat, cen_lon)

19.4011505 -99.169363


In [18]:
cdmx_calor = folium.Map(location=[cen_lat,cen_lon],
                    zoom_start = 12)

#Mapa de calor
lati = []
long = []

coll = db.bici
for doc in coll.find():
    # Simple marker
    lon = doc["lon"]
    lat = doc["lat"]
    lati = lati + [lat]
    long = long + [lon]
    
f_lati = [float(i) for i in lati]
f_long= [float(i) for i in long]

datos_calor = [[la,lo] for la, lo in zip(f_lati, f_long)]
HeatMap(datos_calor, radius = 10).add_to(cdmx_calor)
cdmx_calor

In [14]:
#Estaciones con más de 20 bicies
cdmx_clu = folium.Map(location=[cen_lat,cen_lon],
                    zoom_start = 12)

resp = db.bici.count({"bikes":{"$gte":20}});
disponibles = np.empty([resp,3])
coll = db.bici
counter = 0
for doc in coll.find({"bikes":{"$gte":20}}, {'bikes':1, 'name':1, 'lat':1,'lon':1, '_id':0}):
    lon = doc["lon"]
    lat = doc["lat"]
    num = doc["bikes"]
    disponibles[counter] = [lat, lon, num]
    counter = counter +1

plugins.MarkerCluster(disponibles).add_to(cdmx_clu)

cdmx_clu

C:\Users\pmeji\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """


In [30]:
cdmx_dir = folium.Map(location=[cen_lat,cen_lon],
                    zoom_start = 12)

coll = db.bici
for doc in coll.find({'stationType' : {'$regex' : ".*TPV.*"}}, {'stationType':1, 'name':1, 'lat':1,'lon':1, '_id':0}):
    # Simple marker
    lon = doc["lon"]
    lat = doc["lat"]
    nombre = doc["name"]
    folium.Marker([lat, lon],
    popup= nombre,
    icon=folium.Icon(color='green')
    ).add_to(cdmx_dir)
 

cdmx_dir  

In [36]:
cdmx_tam = folium.Map(location=[cen_lat,cen_lon],
                    zoom_start = 12)
    
coll = db.bici
for doc in coll.find({"slots":{"$gte":20}}, {'slots':1, 'name':1, 'lat':1,'lon':1, '_id':0}) :
    # Simple marker
    lon = doc["lon"]
    lat = doc["lat"]
    nombre = doc["name"]
    tam = doc['slots']

    folium.features.CircleMarker(
    location=[lat,lon],
    radius=tam*10,
    popup=nombre,
    color='red',
    fill_opacity=0.2
    ).add_to(cdmx_tam)
    
cdmx_tam

In [19]:
cdmx_calor.save('cdmx_calor.html')
cdmx_dir.save('cdmx_dir.html')
cdmx_clu.save('cdmx_clu.html')
cdmx_tam.save('cdmx_tam.html')